<a href="https://colab.research.google.com/github/mmistroni/Magentic-AlgoTrading101/blob/main/Restaurant_Menu_Cost_Calculator_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is an Experiment to leverage Multimodal LLM to calculate how much does it cost to dine in Gabicce.
The notebook will load images of restaurants menus and i will ask LLM to calculate how much does it cost for a family of 3 and a standard meal to eat in a restaurant in Gabicce Mare (Italy)
I have downloaded menu images and stored them on my google drive.
I then generate a base64 representation of each image and ask the model to interpret it.

The next step will be to extract structured data from the image, load them
in a vector store and perform some queries

In [1]:
# Mounting the drive

import google.generativeai as genai
import os
import base64
import io
from google.colab import drive

# --- Step 1: Mount Google Drive ---
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.


In [2]:
# --- Step 2: Define the path to your images in Google Drive ---
# IMPORTANT: Update this path to where your images are stored in your Google Drive.
# Example: '/content/drive/MyDrive/MyImagesFolder'
image_folder_path = '/content/drive/MyDrive/Menus' # <--- UPDATE THIS PATH


### Use Cell below to generate base64 representation of your images

In [ ]:
# This cell will generat a base64 so that i can embed all the images in my notebook. you dont need to run it for my example, but if you want to load
# your own images you will need to
# 1. load images in a folder called menus  - see folder on the right
# 2. run the cell below that will output a a base64 image
# 3. copy the output in each new cell
import base64
import os
from IPython.display import Image, display, Markdown

# Define the directory where your images are located
image_directory = image_folder_path#'menus'

# --- Step 1: List all image files in the specified directory ---
image_files = []


encoded_images = []


try:
    # Check if the directory exists
    if not os.path.exists(image_directory):
        print(f"Error: Directory '{image_directory}' not found in the current Colab session.")
        print("Please ensure you have uploaded your 'menus' folder correctly.")
    else:
        # Filter for common image extensions
        for filename in os.listdir(image_directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp')):
                image_files.append(filename)

    if not image_files:
        print(f"No image files found in '{image_directory}'. Please check the folder content.")
    else:
        print(f"Found {len(image_files)} image(s) in '{image_directory}': {', '.join(image_files)}")

except Exception as e:
    print(f"An error occurred while listing files: {e}")

# --- Step 2: Process and output Base64 for each found image ---
if image_files:
    print("\n--- Generating Base64 Markdown for all found images: ---")
    print("-----------------------------------------------------------------")

    for selected_filename in image_files:
        full_image_path = os.path.join(image_directory, selected_filename)

        print(f"\nProcessing '{selected_filename}'...")

        # --- Step 3: Read and Base64 encode the selected image ---
        try:
            with open(full_image_path, 'rb') as img_file:
                encoded_string = base64.b64encode(img_file.read()).decode('utf-8')

            # Determine MIME type based on file extension
            mime_type = "image/png" # Default
            if selected_filename.lower().endswith(('.jpg', '.jpeg')):
                mime_type = "image/jpeg"
            elif selected_filename.lower().endswith('.gif'):
                mime_type = "image/gif"
            elif selected_filename.lower().endswith('.bmp'):
                mime_type = "image/bmp"
            elif selected_filename.lower().endswith('.webp'):
                mime_type = "image/webp"

            # --- Step 4: Generate Markdown for embedding ---
            # You can customize the Alt Text here if needed
            alt_text = f"Image: {selected_filename}"
            markdown_code = f"![{alt_text}](data:{mime_type};base64,{encoded_string})"

            encoded_images.append((selected_filename, mime_type, encoded_string))


        except FileNotFoundError:
            print(f"Error: Image file '{full_image_path}' not found. It might have been deleted.")
        except Exception as e:
            print(f"An error occurred during encoding or display for '{selected_filename}': {e}")
else:
    print("\nNo images available to process. Please upload images to the 'menus' folder.")

from pprint import pprint
pprint(encoded_images)




### Calling model to extract menu from images

In [4]:
from re import M
import base64
import requests
import json
from PIL import Image
import io
from google.colab import userdata

# --- Configuration ---
# Replace with your actual Gemini API Key.
# In a real environment, you'd load this securely (e.g., from environment variables).
# For this Canvas environment, the __api_key__ variable will be provided at runtime.
API_KEY = userdata.get('GOOGLE_API_KEY') # Leave this empty, Canvas will inject the API key.
GEMINI_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

# --- Helper Function: Image to Base64 ---
# --- Function: Extract Menu Items using Gemini API ---
def extract_menu_from_image(base64_image_data, prompt_text):
    """
    Calls the Gemini API to extract menu items and prices from an image.
    It uses a structured response schema to get JSON output.
    """
    if not base64_image_data:
        return {"error": "No image data provided."}

    headers = {
        "Content-Type": "application/json",
    }

    # Define the structured schema for the response
    response_schema = {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "item": {"type": "STRING"},
                "price": {"type": "NUMBER"}
            },
            "required": ["item", "price"]
        }
    }

    payload = {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt_text},
                    {
                        "inlineData": {
                            "mimeType": "image/jpeg", # Adjust mimeType if you save as PNG
                            "data": base64_image_data
                        }
                    }
                ]
            }
        ],
        "generationConfig": {
            "responseMimeType": "application/json",
            "responseSchema": response_schema
        }
    }

    try:
        response = requests.post(f"{GEMINI_API_URL}?key={API_KEY}", headers=headers, data=json.dumps(payload))
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and result['candidates'][0]['content'].get('parts'):
            # The API returns the JSON as a string within the 'text' field
            json_string = result['candidates'][0]['content']['parts'][0]['text']
            # Parse the JSON string into a Python object
            parsed_json = json.loads(json_string)
            return parsed_json
        else:
            print("Unexpected API response structure:", result)
            return {"error": "Could not extract menu items. Unexpected API response."}

    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return {"error": f"API request failed: {e}"}
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON response: {e}")
        print(f"Raw response text: {response.text}")
        return {"error": f"Failed to decode JSON response: {e}"}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": f"An unexpected error occurred: {e}"}

# --- Function: Calculate Meal Cost ---
def calculate_meal_cost(extracted_menu, desired_items, num_people, price_increase_percent, coperto_per_person=2.50):
    """
    Calculates the total estimated cost for a meal based on extracted menu prices,
    desired items, number of people, and a price increase percentage.
    """
    total_food_cost = 0.0
    print("\n--- Calculating Meal Cost ---")
    print(f"Desired items for {num_people} people:")

    for item_name, quantity in desired_items.items():
        found_price = None
        # Try to find the item in the extracted menu (case-insensitive, partial match)
        for menu_item in extracted_menu:
            if item_name.lower() in menu_item['item'].lower():
                found_price = menu_item['price']
                break

        if found_price is not None:
            cost_for_item = found_price * quantity
            total_food_cost += cost_for_item
            print(f"- {quantity}x {item_name}: €{found_price:.2f} each -> €{cost_for_item:.2f}")
        else:
            print(f"- Warning: '{item_name}' not found in the extracted menu. Skipping this item.")
            # For items not found, we'll use a reasonable average from previous context
            # This is a fallback if the OCR/LLM misses something or if the item is generic
            if "patatine fritte" in item_name.lower():
                found_price = 4.50
            elif "insalata" in item_name.lower():
                found_price = 4.50
            elif "primi piatti di pasta" in item_name.lower() or "pasta" in item_name.lower():
                found_price = 15.00
            elif "pizza" in item_name.lower():
                found_price = 10.00
            else:
                found_price = 0.0 # Default to 0 if no reasonable fallback

            if found_price > 0:
                cost_for_item = found_price * quantity
                total_food_cost += cost_for_item
                print(f"  (Using estimated price: €{found_price:.2f} for {item_name} -> €{cost_for_item:.2f})")


    print(f"\nSubtotal for food before increase: €{total_food_cost:.2f}")

    # Apply price increase
    increased_food_cost = total_food_cost * (1 + price_increase_percent / 100)
    print(f"Subtotal for food after {price_increase_percent}% increase: €{increased_food_cost:.2f}")

    # Add coperto
    total_coperto_cost = coperto_per_person * num_people
    print(f"Coperto ({coperto_per_person:.2f} per person for {num_people} people): €{total_coperto_cost:.2f}")

    final_total_cost = increased_food_cost + total_coperto_cost
    print(f"\nEstimated total cost for {num_people} people: €{final_total_cost:.2f}")

    return final_total_cost

# --- Main Execution ---
# --- Step 1: Prepare the Image ---
# We'll use one of the previously uploaded images.
# In a real RAG scenario, you'd retrieve the image data directly from your store.
# For demonstration, ensure 'menu maremosso temporaneo portrait.pages.png' is accessible
# in the same directory as this notebook, or provide its full path.
#image_file_path = "menu maremosso temporaneo portrait.pages.png" # Example image
#print(f"Loading image from: {image_file_path}")
#base64_image = encoded_images[0][2]
menus = {}
for title, typ, base_64_image in encoded_images:
    # --- Step 2: Extract Menu Items using Gemini  need to loop through all imagers---
    print("\n--- Calling Gemini API to extract menu items ---")
    prompt = "Extract all menu items and their prices from this image. Provide the output as a JSON array of objects, where each object has 'item' (string) and 'price' (number) keys. If a price is not explicitly stated, use 0.0. Do not include any introductory or concluding text, just the JSON array."
    extracted_menu_data = extract_menu_from_image(base_64_image, prompt)
    menus[title] = extracted_menu_data


for key, men in menus.items():
  print(f'----------------- {key}-------------')
  from pprint import pprint
  pprint(men)




--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---

--- Calling Gemini API to extract menu items ---
----------------- baiadeibaci.png-------------
[{'item': 'Tortellini al ragu di carne fatto in casa', 'price': 12.0},
 {'item': 'Lasagna al ragù fatta in casa', 'price': 12.0},
 {'item': 'Tagliata di Manzo alla griglia al sale di Cervia e rosmarino',
  'price': 18.0},
 {'item': 'Tagliata di Manzo alla griglia con rucola e pomodorini',
  'price': 19.0},
 {'item': 'Arista di Malale alle mela verde', 'price': 14.0},
 {'item': 'Cotoletta di Tacchino 

### Creating Vector Store

In [ ]:
!pip install chromadb sentence-transformers
!pip install langchain_google_vertexai
!pip install langchain_community
!pip install langchain
!pip install --quiet langchain chromadb sentence-transformers openai langchain-openai

In [6]:
import os
from google.colab import userdata # Import userdata to access Colab Secrets

# Retrieve the OpenAI API Key from Colab Secrets
# The key name here must match the name you set in the Secrets tab (e.g., OPENAI_API_KEY)
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set it as an environment variable for LangChain and OpenAI libraries to pick up
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["CHROMA_OPENAI_API_KEY"] = openai_api_key

In [7]:
import chromadb
from chromadb.utils import embedding_functions
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
CHROMA_DB_PATH = "./chroma_db_data_openai"
COLLECTION_NAME = "restaurant_menus_openai"
# --- 1. Initialize Embedding Functions (for both populating and querying) ---
# For populating ChromaDB directly:
# The OpenAIEmbeddingFunction for ChromaDB's native API will automatically use OPENAI_API_KEY from os.environ
openai_chroma_ef = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-ada-002" # or "text-embedding-3-small", "text-embedding-3-large"
)

# For LangChain to generate query embeddings:
# LangChain's OpenAIEmbeddings wrapper also automatically uses OPENAI_API_KEY from os.environ
langchain_embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002" # Match the model used for population
)



### Approach 1: we will create one document per menu item, to see how it works. Another approach could be to generate one document per menu

In [ ]:

# 3. Prepare texts and metadatas for the vector store
texts = []
metadatas = []
ids = [] # Unique IDs for each entry (optional but good practice)
current_id = 0

for key, restaurant_menu in menus.items():
  restaurant_name = key.split('.')[0]
  print(f'--adding menu for {restaurant_name}-------------')

  # Add Pizza Palace menu items
  for dish in restaurant_menu:
    text_content = f"{dish['item']} - {dish['price']}" # You can choose how to represent the item as text
    texts.append(text_content)
    metadatas.append({"restaurant_name": f"{restaurant_name}", "item_name": dish['item'], "price": dish['price']})
    ids.append(f"{restaurant_name}_{current_id}")
    current_id += 1

# --- 3. Initialize Persistent ChromaDB client and collection (POPULATION PHASE) ---
# Use PersistentClient so the data isn't lost within the Colab session
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# Delete collection if it already exists (for clean re-runs during development)
try:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"Deleted existing collection: {COLLECTION_NAME}")
except Exception as e:
    print(f"Collection '{COLLECTION_NAME}' did not exist or could not be deleted: {e}")
    pass # Collection might not exist yet, so we pass

# Create the collection, passing the SentenceTransformerEmbeddingFunction instance
collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=openai_chroma_ef # Pass the instance here
)

# Add data to the vector store (only if not already populated)
# Checking count() can be a bit slow for very large collections, but fine for this purpose.
if collection.count() == 0: # Check if collection is empty
    collection.add(
        documents=texts,
        metadatas=metadatas,
        ids=ids
    )
    print(f"Successfully added {len(texts)} menu items to the vector store.")
else:
    print(f"Collection '{COLLECTION_NAME}' already contains {collection.count()} items. Skipping re-population.")


print("\n--- ChromaDB Population Complete ---")
print(f"ChromaDB data stored at: {CHROMA_DB_PATH}")



### CHECKING WHAT HAS BEEN STORED

In [ ]:
try:
    collection = client.get_collection(name=COLLECTION_NAME)

    # Retrieve all documents from the collection
    all_documents = collection.get(
        # ids=["id1", "id2"], # You can specify specific IDs if needed
        include=["documents", "metadatas"]
    )

    # Print the retrieved information
    print("Retrieved documents and metadatas:")
    for i, doc in enumerate(all_documents['documents']):
        print(f"--- Document {i+1} ---")
        print(f"Content: {doc}")
        print(f"Metadata: {all_documents['metadatas'][i]}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check that the collection name and persist directory are correct.")


### Kicking off Langchain Pipeline

In [10]:
# --- LangChain Integration (RETRIEVAL & GENERATION PHASE) ---

# 4. Initialize OpenAI LLM
try:
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.3,
    )
    print(f"Successfully initialized OpenAI LLM: {llm.model_name}")
except Exception as e:
    print(f"Error initializing OpenAI LLM. Make sure your OPENAI_API_KEY is correct and accessible. Error: {e}")
    exit()

# 5. Load your ChromaDB collection into LangChain's Chroma wrapper
langchain_chroma_vectorstore = Chroma(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding_function=langchain_embeddings
)
print(f"Successfully loaded ChromaDB collection '{COLLECTION_NAME}' into LangChain.")

# 6. Create a Retriever from the LangChain Chroma vector store
retriever = langchain_chroma_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 100}
)
print("Retriever created.")

Successfully initialized OpenAI LLM: gpt-3.5-turbo
Successfully loaded ChromaDB collection 'restaurant_menus_openai' into LangChain.
Retriever created.


/tmp/ipython-input-2009440212.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  langchain_chroma_vectorstore = Chroma(


In [11]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.combine_documents import create_stuff_documents_chain

# --- 1. Define the formatted document prompt ---
# This is the prompt that will be used to format each individual retrieved document
document_prompt = PromptTemplate.from_template(
    "Restaurant Name: {restaurant_name}\n"
    "Menu Item: {item_name}\n"
    "Price: {price}\n"
    "Content: {page_content}"
)

# --- 2. Define the main RAG prompt for the LLM ---
rag_prompt_template = """You are a helpful assistant that answers questions about restaurant menus.
Use the following pieces of context to answer the user's question.
If the context does not contain the answer, state that you don't know based on the provided information.
Provide the restaurant name, item, and price if available.

Context:
{context}

Question: {input}
Answer:"""

# --- 3. Build the conversational RAG chain with LCEL ---

# 3a. Define the prompt for rephrasing the question
rephrase_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only return the search query and nothing else.")
    ]
)

# 3b. Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(llm, retriever, rephrase_prompt)

# 3c. Define the final RAG chain
qa_chain = create_stuff_documents_chain(
    llm,
    ChatPromptTemplate.from_messages([
        ("system", rag_prompt_template),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]),
    document_prompt=document_prompt
)

# 3d. Combine the two chains into a full retrieval chain
conversational_retrieval_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain
)

print("Conversational retrieval chain with formatted context created.")

# --- 4. Test the new chain with memory and formatted docs ---
print("\n--- Testing Conversational Retrieval Chain ---")

# First query
query1 = "What kind of pizzas do they serve across all restaurants?"
# The chat_history list is empty for the first query
response1 = conversational_retrieval_chain.invoke({"input": query1, "chat_history": []})
print(f"Question: {query1}")
print(f"Answer: {response1['answer']}")
if response1.get('source_documents'):
    print("Source Documents:")
    for doc in response1['source_documents']:
        print(f"  - Content: '{doc.page_content}' | Metadata: {doc.metadata}")
print("-" * 50)

# Follow-up query
query2 = "What about other seafood dishes?"

# The KEY CHANGE is here:
# We now use HumanMessage and AIMessage to build the chat_history
chat_history = [
    HumanMessage(content=query1),
    AIMessage(content=response1["answer"])
]

response2 = conversational_retrieval_chain.invoke({"input": query2, "chat_history": chat_history})

print(f"Question: {query2}")
print(f"Answer: {response2['answer']}")
if response2.get('source_documents'):
    print("Source Documents:")
    for doc in response2['source_documents']:
        print(f"  - Content: '{doc.page_content}' | Metadata: {doc.metadata}")
print("-" * 50)

Conversational retrieval chain with formatted context created.

--- Testing Conversational Retrieval Chain ---
Question: What kind of pizzas do they serve across all restaurants?
Answer: The following pizzas are served across all restaurants mentioned in the provided context:

1. Pizza Margherita - Price: 6.5
2. Pizza Napoletana - Price: 8.0
3. Pizza Marinara - Price: 6.0
4. Pizza 4 Stagioni - Price: 9.0
5. Pizza Romana - Price: 8.0
6. Pizza 4 Formaggi - Price: 9.5
7. Pizza Bufalina - Price: 11.0
8. Pizza Bel Sit - Price: 10.0
9. Pizza Siciliana - Price: 12.0
10. Pizza Ortalana - Price: 10.0
11. Pizza Delicata - Price: 10.5
12. Pizza alla Diavola - Price: 8.0
13. Pizza Mediterranea - Price: 11.0
14. Pizza Parmigiana - Price: 10.0
15. Pizza Mentanara - Price: 11.0
16. Pizza del Pescatore - Price: 11.0
17. Pizza Burratina - Price: 12.0
18. Pizza Stracchino, Rucola, Prosciutto Crudo - Price: 11.0
19. Pizza Salmone - Price: 12.0
20. Pizza Pesto, Gamberi e Pendolini - Price: 12.0
21. Pizza 

In [19]:
query = """What is the average price for a meal with two main courses , a pizza, a patatine fritte, a bottle of water, a beer and an ice cream ? Once you find out, please increase the price by 10% and on top of that add a 12.5% tip
"""

response2 = conversational_retrieval_chain.invoke({"input": query, "chat_history": chat_history})

print(f"Question: {query}")
print(f"Answer: {response2['answer']}")
if response2.get('source_documents'):
    print("Source Documents:")
    for doc in response2['source_documents']:
        print(f"  - Content: '{doc.page_content}' | Metadata: {doc.metadata}")
print("-" * 50)

Question: What is the average price for a meal with two main courses , a pizza, a patatine fritte, a bottle of water, a beer and an ice cream ? Once you find out, please increase the price by 10% and on top of that add a 12.5% tip

Answer: To calculate the average price for the meal with the specified items and apply the requested increases:

1. Two main courses:
   - Average price of main courses: (14.0 + 12.0) / 2 = 13.0

2. Pizza:
   - Average price of pizza: (6.5 + 8.0 + 6.0 + 9.0 + 8.0 + 9.5 + 11.0 + 10.0 + 12.0 + 10.0 + 10.5 + 8.0 + 11.0 + 10.0 + 11.0 + 12.0 + 11.0 + 12.0 + 8.0 + 6.5 + 11.0 + 11.0 + 8.0 + 9.0) / 24 = 9.71

3. Patatine Fritte:
   - Price of Patatine Fritte: 5.0

4. Bottle of water:
   - Average price of a bottle of water: Assumed 2.0

5. Beer:
   - Average price of a beer: Assumed 3.0

6. Ice cream:
   - Average price of an ice cream: Assumed 4.0

Total initial price:
13.0 (main courses) + 9.71 (pizza) + 5.0 (Patatine Fritte) + 2.0 (water) + 3.0 (beer) + 4.0 (ice 

In [16]:
query = """Considering only the most expensive dishes, what is the average price for a meal with two main courses , a pizza, a patatine fritte, a bottle of water and a beer ? Once you find out, please increase the price by 10% and on top of that add a 12.5% tip
"""

response2 = conversational_retrieval_chain.invoke({"input": query, "chat_history": chat_history})

print(f"Question: {query}")
print(f"Answer: {response2['answer']}")
if response2.get('source_documents'):
    print("Source Documents:")
    for doc in response2['source_documents']:
        print(f"  - Content: '{doc.page_content}' | Metadata: {doc.metadata}")
print("-" * 50)

Question: Considering only the most expensive dishes, what is the average price for a meal with two main courses , a pizza, a patatine fritte, a bottle of water and a beer ? Once you find out, please increase the price by 10% and on top of that add a 12.5% tip

Answer: To calculate the average price for a meal with the most expensive dishes, we will consider the following items:

1. Two main courses
2. One pizza
3. Patatine Fritte
4. A bottle of water
5. A beer

The most expensive items for each category are as follows:

1. Two main courses: Tagliata di Contrefiletto con Rucola, Aceto Balsamico e Grana - Price: 22.0 each
2. Pizza: Grigliata mista di pesce alla Romagnola - Price: 24.0
3. Patatine Fritte - Price: 14.0
4. Bottle of water - Price: 2.0
5. Beer - Price: 5.0

Total cost before additional charges:
(2 * 22.0) + 24.0 + 14.0 + 2.0 + 5.0 = 95.0

Adding 10%:
95.0 + (95.0 * 0.10) = 104.5

Adding a 12.5% tip:
104.5 + (104.5 * 0.125) = 117.81

Therefore, the average price for the meal

In [18]:
query = """Could you tell me what is the most expensive restaurant and what is the cheapest, based on the menus you have ?
"""

response2 = conversational_retrieval_chain.invoke({"input": query, "chat_history": chat_history})

print(f"Question: {query}")
print(f"Answer: {response2['answer']}")
if response2.get('source_documents'):
    print("Source Documents:")
    for doc in response2['source_documents']:
        print(f"  - Content: '{doc.page_content}' | Metadata: {doc.metadata}")
print("-" * 50)

Question: Could you tell me what is the most expensive restaurant and what is the cheapest, based on the menus you have ? 

Answer: The most expensive restaurant based on the menus provided is "squero" with the item "TAGLIATA MARCHIGIANA" priced at 22.0.

The cheapest restaurant based on the menus provided is "algrottino" with the item "Verdure Grigliate" priced at 5.0.
--------------------------------------------------
